# Исследование рынка заведений общественного питания в Москве

**Описание проекта**

Вы решили открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Вместе с партнёрами вы решились обратиться к инвесторам. Их интересует текущее положение дел на рынке — сможете ли вы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов?

Вы — гуру аналитики, и партнёры просят вас подготовить исследование рынка. У вас есть открытые данные о заведениях общественного питания в Москве.

## Изучение имеющихся данных

Для начала импортируем необходимые нам библиотеки и посмотрим на наш датафрйм

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import numpy as np
from io import BytesIO
import plotly
import plotly.graph_objs as go

In [ ]:
df = pd.read_csv('/datasets/rest_data.csv')

In [ ]:
df.info()

In [ ]:
df.head(10)

**Итого**

Всего в нашей таблице 15366 строк и шесть столбцов. Типы данных, встречаемых в столбцах: int и object.

Описание данных согласно документации:

*id* — идентификатор объекта;
*object_name* — название объекта общественного питания;
*chain* — сетевой ресторан;
*object_type* — тип объекта общественного питания;
*address* — адрес;
*number* — количество посадочных мест.

В названиях колонок нет нарушений стиля. Количество значений в столбцах сопадает с общим количеством строк в датафрейме. Это значит, в данных скорее всего отсутствуют пропущенные значения. Типы данных встречаемых в столбцах - адекватные. Для полноты картины необходимо ещё рассмотреть наличие дубликатов.

## Предобработка данных

Убедимся что пропущенных значений действительно нет

In [ ]:
df.isna().mean()

### Поищем дубликаты

Посчитаем количество явных дубликатов. Сначала приведём все значения с типом object к единому нижнему регистру

In [ ]:
df['object_name'] = df['object_name'].str.lower()
df['chain'] = df['chain'].str.lower()
df['object_type'] = df['object_type'].str.lower()
df['address'] = df['address'].str.lower()

Посчитаем количество явных дубликатов

In [ ]:
df.duplicated().sum()

А что если убрать уникальный столбец id? Посмотрим на количество дубликатов в таком разрезе

In [ ]:
print("Кол-во дубликатов: {}".format(df.duplicated(subset=['object_name', 'chain', 'object_type', 'address', 'number']).sum()))

Нашлись таки, дубликатики. Однако 85 дублей - количество не значительное. Поэтому мы просто избавимся от них, не рискуя исказить данные.

In [ ]:
df = df.drop_duplicates(subset = ['object_name', 'chain', 'object_type', 'address', 'number'])

Проверим теперь

In [ ]:
print("Кол-во дубликатов: {}".format(df.duplicated(subset = ['object_name', 'chain', 'object_type', 'address', 'number']).sum()))

**Итого**

Мы убедились в отсутсвии пропущенных значений. А также нашли и избавились от незначительного числа дубликатов.

Теперь наш датафрейм готов к использованию, чтобы мы смогли ответить на большинство интересующих нас вопросов, касаемо расположения заведений общественного питания в Москве. После чего можно будет рекомендовать заказчику выбрать наиболее подходящее место для открытия своего оригинального кафе.

## Отвечаем на вопросы:

Зададим размеры для будущих графиков

In [ ]:
sns.set(rc={'figure.figsize':(10,7)})

И определим палитру для графиков

In [ ]:
sns.set_palette('bright') 

### Исследуем соотношение видов объектов общественного питания по количеству

Посмотрим, какие вообще у нас есть виды объектов общественного питания

In [ ]:
df['object_type'].value_counts()

Изменим значения в столбце на более читабельные

In [ ]:
df = df.replace({'object_type': {'предприятие быстрого обслуживания': 'фастфуд',
                                 'магазин (отдел кулинарии)': 'кулинария'}})

In [ ]:
df['object_type'].value_counts()

Теперь построим функцию которая будет рисовать подписи к графикам, принимать словарь с подписями и выводить их на экран

In [ ]:
def get_labels_and_axes(dic):
    plt.xlabel(dic['xlabel'])
    plt.ylabel(dic['ylabel'])
    plt.title(dic['title'])

Теперь можем строить график соотношения видов объектов общественного питания по количеству

In [ ]:
sns.countplot(y = 'object_type', data = df)
titles = {'xlabel':'Количество заведений', 'ylabel':'Вид заведения',
          'title': 'Соотношение видов объектов общественного питания по количеству'
         }
get_labels_and_axes(titles)
plt.show()

**Вывод**

Самый распространенный формат заведений в Москве - кафе. Из этого следует, что это формат с самым высоким количеством конкурентов. В тоже время можно предположить, что если количество заведений большое, то у потребителя есть спрос на данный формат объектов общественного питания

### Исследуем соотношение сетевых и несетевых заведений по количеству

Изучим общее количество сетевых и несетевых заведений

In [ ]:
df['chain'].value_counts()

Поменяем значения на более понятные

In [ ]:
df = df.replace({'chain' : { 'нет': 'одиночное', 'да': 'сетевое'}})

In [ ]:
df['chain'].value_counts()

Теперь построим график

In [ ]:
sns.countplot(x = 'chain', data = df)
titles = {'xlabel':'Статус заведения', 'ylabel':'Количество', 'title': 'Соотношение сетевых и несетевых заведений по количеству'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Несетевых заведений значительно больше. Думаю, это можно связать с тем, что далеко не каждое заведение общественного питания доходит до тех значений выручки, с которыми можно расширяться до сети. Да и не каждый ресторанный бизнес имеет амбиции по расширению путём открытия новых филиалов

### Ответим на вопрос: Для какого вида объекта общественного питания характерно сетевое распространение

Построим сводную таблицу, а также график к ней, чтобы увидеть процент сетевых заведений по каждому типу

In [ ]:
chain_object_type = (df
 .pivot_table(index='object_type', columns = 'chain', values = 'id', aggfunc = 'count')
 .reset_index()
 .assign(part = lambda x: x['сетевое'] / (x['сетевое']+x['одиночное']))
 .sort_values(by = 'part', ascending=False)
)

In [ ]:
chain_object_type.style.format(formatter={'part':'{:.2%}'})

In [ ]:
chain_objects_plot = sns.barplot(x = 'part', y = 'object_type', data=chain_object_type, palette = 'Paired')
vals = chain_objects_plot.get_xticks()
titles = {'xlabel':'Доля сетевых заведений', 
          'ylabel':'Вид объекта общественного питания', 
          'title': 'Доли сетевых заведений по разным видам объектов общественного питания'}
get_labels_and_axes(titles)
chain_objects_plot.set_xticklabels(['{:,.2%}'.format(x) for x in vals])
plt.show()

**Вывод**

Видим, что большая доля сетевых заведений встречается в предприятиях в простонародье именуемых *фастфуды*. Что логично, ведь для для этих заведений характерна дешевизна и чётко отработанная система взаимодействия сотрудников. Ниличие этих качеств и не зациклненность на других моментах ресторанного бизнеса, позволяет открывать почти одинаковые точки питания сразу во многих местах. А скорость приготовления и дешевизна обеспечит большой приток клиентов, что в свою очередь породит ещё большее расширение и открытие новых точек. *Пока фастфуды не заполонят собой всю вселенную(Но это не точно)*

### Какое распределение количества посадочных мест характерно для сетевых и несетевых заведений

Для ответа на этот вопрос построим график распределение количества посадочных мест для сетевых и несетевых заведений в зависимости от вида заведения

In [ ]:
sns.stripplot(x = 'number', y = 'object_type', data = df, hue = 'chain', dodge = True)
titles = {'xlabel':'Количество посадочных мест', 
          'ylabel':'Вид заведения', 
          'title': 'Распределение количества посадочных мест для сетевых и несетевых заведений'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Для кафе распределение количества посадочных мест для сетевых и несетевых заведений практически совпадает.

Исходя их этого можем сделать вывод, что при открытии кафе следует ориентироваться на среднее количество посадочных мест характерное для данного формата заведения общественного питания.

### Зависимость количества посадочных мест по видам объектов

In [ ]:
sns.barplot(x = 'number', y = 'object_type', data = df.sort_values('object_type'))
titles = {'xlabel':'Среднее количество посадочных мест', 
          'ylabel':'Вид объекта', 
          'title': 'Среднее количество посадочных мест по виду объекта'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Больше всего посадочных мест в столовых. Что не удивительно, ведь столовые рассчитаны на обслуживание большого количества человек в обеденный перерыв.

Меньше всего мест в кулинариях, закусочных и кафетериях. Все эти заведения как правило имеют доволльно скромные площади и направлены на достаточно узкий круг клиентов(покупатели в магазинах, при которых находятся кулинарии; закусочные рядом с заправками; кафетерии в аэропорту). То есть, как правило, такие заведения не являются местами, куда люди целенаправленно идут покушать, а скорее заглядывают перекусить по дороге.

Что примечательно, для предприятий фастфуда также характерно небольшое количество мест. Так происходит из за того, что формат быстрого питания предполагает постоянный трафик клиентов. К тому же в подобных заведениях пользуется спросом функция *"еда на вынос"*

### Улицы

#### Выделим отдельный столбец под название улиц

Возьмём столбец *address* и выделим из него названия отдельных улиц в столбец *street*

In [ ]:
df = df.assign(street = lambda x: x['address'].str.split(',')
               .apply(lambda x: x[1] if 'город' not in x[1] and
               'поселение' not in x[1] else (
                   x[2] if 'корпус' not in x[2] else 'нет улицы')))

Посмотрим что получилось

In [ ]:
df.head(15)

#### Рассмотрим топ-10 улиц с наибольшим количеством объектов

Создадим отдельный датафрейм, где посчитаем количество заведений на каждой из улиц

In [ ]:
df_top_streets = (
    df.loc[df['street'] != 'нет улицы'].groupby(['street'])
    .agg(amount = ('id','count'))
    .sort_values(by = 'amount', ascending=False)
    .reset_index()
    .loc[:10]
)

При этом отсортировав по убыванию и оставив только топ-10. Значения "нет улицы" исключим, чтобы не мешало расчётам.

In [ ]:
df_top_streets

In [ ]:
sns.barplot(y = 'street', x = 'amount', data = df_top_streets)
titles = {'xlabel':'Количество заведений', 
          'ylabel':'Улица', 
          'title': 'Топ 10 улиц по количеству заведений'}
get_labels_and_axes(titles)
plt.show()

#### Посмотрим в каких районах Москвы расположены эти улицы

Для начала создадим отдельный список с наиболее популярными улицами

In [ ]:
df_streets_top_list = df_top_streets['street'].to_list()

In [ ]:
df_streets_top_list

Создадим и отфильтруем датафрейм согласно списка этих улиц

In [ ]:
df_filtered = df[df['street'].isin(df_streets_top_list)]

In [ ]:
df_filtered

Теперь создадим датафрейм с уникальными адресами

In [ ]:
df_addresses = pd.DataFrame({'unique_address' : df_filtered['address'].unique()})

In [ ]:
df_addresses

Создадим функцию, которая получит для нас координаты через API Яндекс.Карт и прочитаем датафрейм из файла

In [ ]:
def get_coordinates(address):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    response = requests.get(base_url, params={
        "geocode": address,
        "apikey": '42096618-c47f-45f8-bca4-2c6eb2cbd672',
        "format": "json",
    })
    response.raise_for_status()
    found_places = response.json()['response']['GeoObjectCollection']['featureMember']

    if not found_places:
        return None

    most_relevant = found_places[0]
    coordinates = most_relevant['GeoObject']['Point']['pos'].split(' ')
    return coordinates

In [ ]:
url = 'https://drive.google.com/file/d/1LworkYMR8ikIZW3Wbvlii0fzII0x1ntZ/view?usp=sharing'
path = 'https://drive.google.com/uc?id='+url.split('/')[-2]
df_addresses = pd.read_csv(path, sep=';')

Теперь нам понадобится функция поиска района

In [ ]:
def get_district(coordinates):   
    base_url = "https://geocode-maps.yandex.ru/1.x"
    response = requests.get(base_url, params={
        "geocode": coordinates[0] + ',' + coordinates[1],
        "apikey": '42096618-c47f-45f8-bca4-2c6eb2cbd672',
        "format": "xml",
        'kind': 'district'
    })
    soup = BeautifulSoup(response.text, 'xml')
    result = soup.find_all('DependentLocalityName')
    for item in result:
        if 'район' in item.text:
            return item.text

Прочитаем датафрейм из файла и приведём столбец, по которому будем объединять, к единому названию. После этого можем спокойно объединять наши датафреймы

In [ ]:
url = 'https://drive.google.com/file/d/1jEi1FxGGj75en3XE_vZ-FShxdRHInY0H/view?usp=sharing'
path = 'https://drive.google.com/uc?id='+url.split('/')[-2]
df_addresses_and_districts = pd.read_csv(path, sep=';')

In [ ]:
df_addresses_and_districts.rename(columns={"unique_address": "address"},
                                  inplace=True)

In [ ]:
df_filtered = df_filtered.merge(df_addresses_and_districts, on='address')

In [ ]:
df_filtered

Дальше группируем по улицам и районам

In [ ]:
df_objects_by_districts_and_streets = (
    df_filtered.groupby(['street', 'district'])
    .agg(objects = ('id','count'))
    .reset_index()
)

In [ ]:
df_objects_by_districts_and_streets

In [ ]:
df_objects_by_districts = (
    df_filtered.groupby(['district'])
    .agg(objects=('id', 'count'))
    .sort_values(by='objects', ascending=False)
    .reset_index()
)

In [ ]:
df_objects_by_districts

И строим график количества объектов по районам

In [ ]:
sns.barplot(y='district', x='objects', data=df_objects_by_districts)
titles = {'xlabel':'Количество заведений', 
          'ylabel':'Район', 
          'title': 'Количество объектов в районе'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Видим, что больше всего заведений у нас в Пресненском районе. Что не удивительно, ведь в нём расположены Москва-Сити и Экспоцентр(популярные места для туристов, к тому же большое количество офисных работников, которые ходят на обед). 
Следом с заметным отрывом идут Хороёвский район, Чертаново Центральное и Тропарёво-Никулино. В дальнейшем ситуация сглаживается и уже более плавно идёт на уменьшение по количеству объектов общественного питания

#### Теперь зайдём с другой стороны и посмотрим сколько улиц с единственным объектом общественного питания и где они находятся

По аналогии с предыдущим запросов, создадим датафрейм с уникальными адресами

In [ ]:
df_streets_with_one_object = (
    df.groupby(['street'])
    .agg(amount=('id','count'))
    .sort_values(by='amount', ascending=False)
    .reset_index()
    .loc[lambda x: x['amount'] == 1]
    .sort_values(by='street')
)

In [ ]:
df_streets_with_one_object

Наблюдаем целых 629 улиц. И все они с одним единственным объектом общественного питания

Теперь создадим список с этими улицами и отфильтруем по ним наш общий датафрейм. Это необходимо для того, чтобы заняться поиском координат по полному адресу(т.к. одна улица может находиться в двух или более районах).

In [ ]:
df_streets_with_one_object_list = (
    df_streets_with_one_object['street'].to_list()
)

In [ ]:
df_streets_with_one_object_list

In [ ]:
df_filtered_with_one_object = (
    df[df['street'].isin(df_streets_with_one_object_list)]
)

In [ ]:
df_filtered_with_one_object

In [ ]:
url = 'https://drive.google.com/file/d/1mTbtFblYKELOF7kLWp5mcNUxzAtLXCjI/view?usp=sharing'
path = 'https://drive.google.com/uc?id='+url.split('/')[-2]
df_filtered_with_one_object = pd.read_csv(path, sep=';')

Разобём строку с адресом и присвоим району второй элемент

In [ ]:
df_filtered_with_one_object_districts['district'] = (
    df_filtered_with_one_object_districts['district']
    .fillna(df_filtered_with_one_object_districts['address']
    .str.split(',').apply(lambda x: x[1]))
)

Дальше сгруппируем и отдельно выведем топ-10

In [ ]:
df_one_object_by_districts = (
    df_filtered_with_one_object_districts.groupby(['district'])
    .agg(objects=('id', 'count'))
    .sort_values(by='objects', ascending=False)
    .reset_index()
)

In [ ]:
df_one_object_by_districts_top = df_one_object_by_districts.loc[:10]

In [ ]:
df_one_object_by_districts_top

In [ ]:
sns.barplot(y='district', x='objects', data=df_one_object_by_districts_top)
titles = {'xlabel':'Количество заведений', 
          'ylabel':'Район', 
          'title': 'Количество объектов в районе'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Улиц с одним едиснтвенным заведением общественного питания довольно много(больше 600). Снова видим в лидерах районы, которые расположены близко к центру Москвы: Таганский, Хамовники, Басманный, Тверской. В прочем, это не удитвительно. Ведь Москва старый город, а значит что в центре имеется огромное количество маленьких улочек и переулков, на которых и находятся эти едиственные "на всю улицу" заведения общественного питания.

#### Теперь посмотрим на распределение количества посадочных мест для улиц с большим количеством объектов общественного питания

Для этого создадим функцию, которая принимает на вход адрес, разделяет на несколько подстрок и проверяет каждую подстроку на наличе в ней информации об улице

In [ ]:
def get_street(address):
    street_list = ['улица', 'проспект', 'набережная', 
               'проезд', 'бульвар', 'шоссе', 
               'переулок', 'площадь', 'аллея', 
               'квартал', 'линия', 'тупик']
    for item in address.split(','):
        for string in street_list:
            if string in item:
                return item.strip()
    return 'Not found'          

Создадим список для удобства построения графика

In [ ]:
street_list = df_top_streets['street'].to_list()

И посмотрим как выглядит распределение количества посадочных мест для улиц с большим количеством заведений

In [ ]:
sns.distplot(df.loc[lambda x: x['street'].isin(street_list)]['number'], 
             kde=False,
             bins = 100)
plt.xlim(0,400)
titles = {'xlabel':'Количество посадочных мест', 
          'ylabel':'Количество заведений', 
          'title': 'Распределение количества посадочных мест для улиц с большим количеством заведений'}
get_labels_and_axes(titles)
plt.show()

**Вывод**

Господствуют заведения c количеством посадочных мест от 1 до 50. Это похоже на распределение количества посадочных мест для сетевых заведений, которое мы искали ранее. Затем идут объекты, где от 51 до 100 посадочных мест. И замыкают этот список заведения без посадки(видимо, они работают только на вынос).

## Общий вывод

Мы изучили рынок заведений общественного питания в Москве и установили, **что:**
- Самый распространенный формат заведений в Москве - кафе;
- Количество одиночных заведений гораздо больше их сетевых собратьев;
- Большая доля сетевых заведений встречается в предприятиях быстрого питания (они же - фастфуды);
- Для кафе распределение количества посадочных мест для сетевых и несетевых заведений практически совпадает;
- Наибольшим количтевом посадочных мест обладают заведния типа *столовая* и *ресторан*, а наименьшим - *кулинария* и *закусочная*.

А так же мы нашли топ-10 улиц по количеству заведений общественного питания на них, определили в каких районах их больше всего; нашли улицы всего с одним заведением и определили их пристутсвие в тех или иных районах. И в добавок, посмотрели на распределение количества посадочных мест для улиц с большим количеством заведений.

**Исходя из нашего исследования, можно сделать некоторые рекомендации:**

1. Наиболее подходящим форматом при отрытии нового заведения является кафе;

2. По количеству посадочных мест следует ориентироваться на среднее значение для кафе (около 40 мест);

3. Территориально определить оптимальное место сложно, т.к. мы не имеем информации о наличии и финансовой доступности конкретных плоащдей. Однако, кажется очевидным, что это должны быть районы в центральной части Москвы, т.к. большой туричтический трафик и наличие офисных зданий могут стать основой успешной работы завдения общественного питания.